In [ ]:
# ------------- WEEK 1 -------------
#1. Introduction/Business Problem: I was commissioned by the CEO to find a new location for a bank in Toronto. However, I am from New York and do not know the neighborhoods in Toronto. The location should be chosen in such a way that no other bank is located in the neighborhood and a high customer frequency can be expected. A selection of 5 possible locations should be the result.
#2. The Foursquare location data from Toronto is used. These are used to find out if there is already a bank in the neighborhood. In addition, the data enables the identification of neighborhoods with many ventures. A location should be determined by combining both information.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from bs4 import BeautifulSoup

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [39]:
# download data and parse it:
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(r.text, 'html.parser')
table=soup.find('table', attrs={'class':'wikitable sortable'})

#make lists for columns
A=[]
B=[]
C=[]

#go through rows and fill lists
for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True).rstrip())
        B.append(cells[1].find(text=True).rstrip())
        C.append(cells[2].find(text=True).rstrip())

#build dataframe with lists
df=pd.DataFrame(A,columns=['Post Code'])
df['Borough']=B
df['Neighborhood']=C

# drop not assigned boroughs
df = df.drop(df[(df.Borough == "Not assigned")].index)
# give "Not assigned" Neighborhoods same name as Borough
df.Neighborhood.replace("Not assigned", df.Borough, inplace=True)

df.shape

(103, 3)

In [42]:
#add geo-data
dfgeo= pd.read_csv("http://cocl.us/Geospatial_data")
dfgeo.rename(columns={'Postal Code':'Post Code'}, inplace=True)
dfgeo.set_index("Post Code")
df.set_index("Post Code")
toronto_data=pd.merge(df, dfgeo)
toronto_data.head()

,Post Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [44]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto, ON, Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto, ON, Canada are 43.6534817, -79.3839347.


In [80]:
# create map of Toronto, Canada using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto



In [73]:
CLIENT_ID = '532KKFQ4NDT0AAJHBMX0MMQD4FOL5PJKK2RTT0A4G1FYLQCW' # your Foursquare ID
CLIENT_SECRET = 'PLGEINWZYOWGJWTVR0AH2OH0X4R2MB3OT5LHHXSGRC232GR3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 532KKFQ4NDT0AAJHBMX0MMQD4FOL5PJKK2RTT0A4G1FYLQCW
CLIENT_SECRET:PLGEINWZYOWGJWTVR0AH2OH0X4R2MB3OT5LHHXSGRC232GR3


In [78]:
#explore the 4th neighborhood
neighborhood_latitude = toronto_data.loc[4, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[4, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[4, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

#top 100 venues that are within a radius of 500 meters.
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

#get results
results = requests.get(url).json()
results

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

    
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON


# structure it into a pandas dataframe
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]# filter columns

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)# filter the category for each row

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]# clean columns


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#dataframe 
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

# count venues for each neighborhood
toronto_venues.groupby('Neighborhood').count()

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

#group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

#find all neighborhoods without a bank
toronto_bank = toronto_grouped.loc[toronto_grouped['Bank'] == 0]

#sort neighborhoods by number of venues
toronto_venues_ranking = toronto_venues.groupby('Neighborhood').count()
venue_ranking_sorted = toronto_venues_ranking.sort_values(by=["Venue"], ascending=False)

#find 5 neighborhoods with no banks and highest amount of venues
s1 = pd.merge(venue_ranking_sorted, toronto_bank, how='right', on=['Neighborhood'])
s2 = s1.drop(columns=['Neighborhood Latitude', 'Neighborhood Longitude','Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category' ])
s3 = s2[:5]#dataframe of 5 most valuable neighborhoods for a new bank

num_top_venues = 5

for hood in s3['Neighborhood']:
    print("Choice of location for a new bank in descending order")
    print("----"+hood+"----")
    temp = s3[s3['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

Latitude and longitude values of Queen's Park, Ontario Provincial Government are 43.6623015, -79.3894938.


/Users/andreas/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


Choice of location for a new bank in descending order
----Toronto Dominion Centre, Design Exchange----
                venue  freq
0         Coffee Shop  0.12
1               Hotel  0.08
2                Café  0.07
3          Restaurant  0.04
4  Italian Restaurant  0.03


Choice of location for a new bank in descending order
----Harbourfront East, Union Station, Toronto Islands----
         venue  freq
0  Coffee Shop  0.13
1     Aquarium  0.05
2        Hotel  0.04
3         Café  0.04
4   Restaurant  0.03


Choice of location for a new bank in descending order
----Richmond, Adelaide, King----
            venue  freq
0     Coffee Shop  0.10
1            Café  0.05
2  Clothing Store  0.04
3             Gym  0.04
4           Hotel  0.04


Choice of location for a new bank in descending order
----First Canadian Place, Underground city----
                 venue  freq
0          Coffee Shop  0.10
1                 Café  0.08
2                Hotel  0.05
3           Restaurant  0.04
4  Japan

In [ ]:
# ------------- WEEK 2 -------------
#3. Methodology: The known data from week 3 of the Capstone project were used. From this data neighborhoods without a bank were extracted. In a second step, the neighborhoods of Toronto were identified according to the decreasing number of venues. These data were used to determine a ranking for the site selection (locations with many venues were preferred). 
#4. Results: As requested by the CEO, 5 locations have been identified as candidates for a new location, as they do not yet have a bank and customer traffic is expected to be high due to many local businesses. To enable the CEO to make an objective decision, the top 5 business types in the neighbourhood were identified. 
#5. Discussion: The data analysis gives a good first impression which location could be suitable. For a closer look it would be necessary to find out which types of business are frequently visited in combination with a bank. This would be possible with a similar approach. 
#6. Conclusion: There are 5 possible locations that should bring the greatest benefits to our bank. In an upcoming discussion with the CEO, a decision will either be made on the basis of the top 5 businesses at the respective location or alternatively further parameters will be determined which are decisive for our choice.